# Note
## Jun. 6
>Status
1. By now, the best regressor is simple LassoCV using CV = 5
2. For categorical feature, use OneHot encoder.
3. The features having unique values, are dropped.
4. Try box-cox change of Y, but no imporvement.

>TODO
1. More work on feature engineering, try to reduce the dimension:
 + PCA
 + delete the dimensions with small variance

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_y = pd.read_pickle('train_y.pkl')
train_x = pd.read_pickle('train_x.pkl')
test_x =  pd.read_pickle('test_x.pkl')

# Regression
## Xgboost

In [23]:
from sklearn.metrics import r2_score
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

def xgb_cv_score(params, X, Y, n_folds = 3):
    kf = KFold(n_splits=n_folds)
    train_score = list()
    test_score = list()
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        train_d = xgb.DMatrix(X_train, y_train)
        test_d = xgb.DMatrix(X_test)
        print '-'*8 + 'START training loop' + '-'*8
        xgb_reg = xgb.train(dict(params, silent=0), train_d, num_boost_round=1000)
        print '-'*8 + 'Training loop DONE' + '-'*8
        train_score.append(
            r2_score(train_d.get_label(), xgb_reg.predict(train_d)))
        test_score.append(
            r2_score(y_test, xgb_reg.predict(test_d)))
    print train_score, test_score
    return np.mean(train_score), np.mean(test_score)

In [24]:
xgb_params = {
    'n_trees': 500, 
    'eta': .01,
    'max_depth': 2,
    'subsample': 0.8,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': train_y.mean(), # base prediction = mean(target)
    'silent': 0,
    'alpha': 100,
    'lambda':0,
    'gamma':0.5
}
xgb_cv_score(xgb_params, train_x, train_y)

--------START training loop--------
--------Training loop DONE--------
--------START training loop--------
--------Training loop DONE--------
--------START training loop--------
--------Training loop DONE--------
[0.61652872922899893, 0.58185621001990062, 0.5720455543858276] [0.49746142889296496, 0.58412668414617319, 0.59312101395527583]


(0.59014349787824238, 0.55823637566480466)

In [30]:
xgb_reg = xgb.XGBRegressor(
    max_depth=4, 
    learning_rate = 0.01, 
    n_estimators= 500,
    objective='reg:linear',
    subsample=0.8, 
    #colsample_bytree = 0.618,
    gamma=0.5,
    base_score = train_y.mean(),
    reg_alpha= 100, 
    reg_lambda=0,
    n_jobs=-1)
params ={'max_depth':[2,3,4], 'gamma':[0.5]}
xgb_regcv = GridSearchCV(xgb_reg, params, verbose=5)
#xgb_regcv.fit(train_x, train_y)

In [26]:
xgb_regcv.grid_scores_

E:\Anaconda2\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.55588, std: 0.04261, params: {'max_depth': 2, 'gamma': 0.5},
 mean: 0.56126, std: 0.03938, params: {'max_depth': 3, 'gamma': 0.5},
 mean: 0.55992, std: 0.03874, params: {'max_depth': 4, 'gamma': 0.5}]

In [27]:
xgb_reg = xgb_regcv.best_estimator_
xgb_regcv.best_params_

{'gamma': 0.5, 'max_depth': 3}

In [31]:
xgb_reg.fit(train_x, train_y)

XGBRegressor(base_score=100.669318128, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.5, learning_rate=0.01, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=-1, nthread=-1, objective='reg:linear', random_state=0,
       reg_alpha=100, reg_lambda=0, scale_pos_weight=1, seed=0,
       silent=True, subsample=0.8)

In [32]:
test_pred = xgb_reg.predict(test_x)
pd.DataFrame({'ID':test_df.ID,'y':test_pred}).to_csv('res_xgb.csv',index =False)